In [1]:
import pandas as pd
import random
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import glob
import os
import cv2
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from numpy import expand_dims, argmax
import keras
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import resnet50, inception_v3, vgg16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow import keras
from keras import optimizers
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from keras.models import Sequential, load_model, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB4
from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import label_binarize

scaler = MinMaxScaler()

%matplotlib inline

### **Reading data in proper format**

In [2]:
df = pd.read_csv('../input/csv-mod-dataset/Module_1 - Rooftop_Detectection_Mod_Dataset.csv')

### **Label Encoding**

In [3]:
label_encoder = preprocessing.LabelEncoder()
df['Type_LabelEncoded'] = label_encoder.fit_transform(df['Type'])

# Complex -> 0
# Flat -> 1
# Gable -> 2
# Hip -> 3
# Pyramid -> 4 

In [4]:
df['Type'].value_counts()

In [5]:
ax = df['Type'].value_counts().plot(kind = 'bar',
                                    figsize = (14,8),
                                    title = "No of rooftops for each type",
                                    color = 'orange')
ax.set_xlabel("Rooftop Type")
ax.set_ylabel("Frequency")

### **Storing image names in list**

In [6]:
path = '../input/rooftype-dataset-mod/rooftype-dataset-mod-jpg'
image_names = []
labels = []

for img in glob.glob(os.path.join(path, 'Flat-jpg', '*')):
    image_names.append(img)
    labels.append(0)
    
for img in glob.glob(os.path.join(path, 'Gable-jpg', '*')):
    image_names.append(img)
    labels.append(1)
    
for img in glob.glob(os.path.join(path, 'Hip-jpg', '*')):
    image_names.append(img)
    labels.append(2)

print("No of images: {0}".format(len(image_names)))
print("Labels: {0}".format(len(labels)))

### **Shuffle list**

In [7]:
zipped_list = list(zip(image_names, labels))
random.shuffle(zipped_list)
image_names, labels = zip(*zipped_list)
image_names, labels = list(image_names), list(labels)

print(image_names)
print(labels)

### **Pre-processing**

In [8]:
X = []
Y = []
for img in image_names:
    img = cv2.imread(img, cv2.IMREAD_COLOR)
    resized_img = cv2.resize(img, (256, 256), interpolation = cv2.INTER_CUBIC)
    X.append(resized_img)

Y = labels

In [9]:
X = np.array(X)
Y = np.array(Y)

print(X.shape, Y.shape)

In [10]:
X = X.astype('float32')
X /= 255.
Y = Y.reshape(Y.shape[0],1)
print(X.shape, Y.shape)

### **Train-test split**

In [11]:
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size = 0.04)
train_X, val_X, train_Y, val_Y = train_test_split(train_X, train_Y, test_size = 0.05)

In [12]:
print("Train X: {0}, Train Y: {1}".format(train_X.shape, train_Y.shape))
print("Val X: {0}, Val Y: {1}".format(val_X.shape, val_Y.shape))
print("Test X: {0}, Test Y: {1}".format(test_X.shape, test_Y.shape))

### **One-hot encoding**

In [13]:
train_Y = to_categorical(train_Y, num_classes = 3)
val_Y = to_categorical(val_Y, num_classes = 3)
test_Y = to_categorical(test_Y, num_classes = 3)

print("Shape: {0}, {1}, {2}".format(train_Y.shape, val_Y.shape, test_Y.shape))

### **Plotting images**

In [14]:
rows = 36
columns = 3
count = 1
for image_number in range(0,50, 3):

    fig = plt.figure(figsize=(15,15))

    fig.add_subplot(rows, columns, count)
    plt.subplot(1,3,1)
    plt.imshow(np.squeeze(train_X[image_number:image_number+1]))

    fig.add_subplot(rows, columns, count+1)
    plt.subplot(1,3,2)
    plt.imshow(np.squeeze(train_X[image_number+1:image_number+2]))

    fig.add_subplot(rows, columns, count+2)
    plt.subplot(1,3,3)
    plt.imshow(np.squeeze(train_X[image_number+2:image_number+3]))

    count += 3


### **Data Augmentation**

In [15]:
data_generation = ImageDataGenerator(
    rotation_range = 7,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range = 0.10,  # randomly shift images horizontally (fraction of total width)
    height_shift_range = 0.10,  # randomly shift images vertically (fraction of total height)
    horizontal_flip = True,  # randomly flip images
    vertical_flip = True,   # randomly flip images
    fill_mode = 'reflect')  
data_generation.fit(train_X)

### **Plot graphs**

In [16]:
def plot_accuracy_loss(history):
    """
        Plot the accuracy and the loss during the training of the nn.
    """
    fig = plt.figure(figsize=(10,5))

    # Plot accuracy
    plt.subplot(221)
    plt.plot(history.history['acc'], label = "acc")
    plt.plot(history.history['val_acc'], label = "val_acc")
    plt.title("train_acc vs val_acc")
    plt.ylabel("accuracy")
    plt.xlabel("epochs")
    plt.legend()

    # Plot loss function
    plt.subplot(222)
    plt.plot(history.history['loss'], label = "loss")
    plt.plot(history.history['val_loss'], label = "val_loss")
    plt.title("train_loss vs val_loss")
    plt.ylabel("loss")
    plt.xlabel("epochs")

    plt.legend()
    plt.show()


### **Make Predictions**

In [17]:
def make_predictions(model, test_X, test_Y):
    test_loss = model.evaluate(test_X, test_Y)
    predictions = model.predict(test_X)
    pred_labels = np.argmax(predictions, axis = 1)
    test_labels = np.argmax(test_Y, axis = 1)
    return pred_labels, test_labels

### **Plot Confusion Matrix**

In [18]:
def plot_confusion_matrix(test_labels, pred_labels):
    Y_test_actual = test_labels
    Y_test_pred = pred_labels
    confusion_mtx = confusion_matrix(Y_test_actual, Y_test_pred) 

    f,ax = plt.subplots(figsize = (8, 8))
    sns.heatmap(confusion_mtx, annot = True, linewidths = 0.01, cmap = "Greens", linecolor = "gray", fmt = '.1f', ax = ax)
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title("Confusion Matrix")
    plt.show()

### **Classification Report**

In [19]:
def print_classification_report(test_labels, pred_labels):
    print("Classification Report: ")
    print(classification_report(test_labels, pred_labels))

### **Plot AUC-ROC Graph**

In [20]:
def plot_AUC_ROC(model, test_X, test_Y):

    predictions = model.predict(test_X) 
    print(predictions)

    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(3):
        fpr[i], tpr[i], _ = roc_curve(test_Y[:, i], predictions[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    print("ROC_AUC score for 3 models: {0}".format(roc_auc))

    # Plot of a ROC curve for a specific class
    for i in range(3):
        plt.figure()
        plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic example')
        plt.legend(loc="lower right")
        plt.show()

### **Validating our model on PotsDam dataset**

In [21]:
potsdam_image_paths_list = []
potsdam_image_labels = []
potsdam_image_path_flat = '../input/potsdam-mini/potsdam-mini/Flat'   # 0
potsdam_image_path_gable = '../input/potsdam-mini/potsdam-mini/Gable' # 1
potsdam_image_path_hip = '../input/potsdam-mini/potsdam-mini/Hip'     # 2



for img_path in glob.glob(os.path.join(potsdam_image_path_flat, '*.tif')):
    potsdam_image_paths_list.append(str(img_path)) 
    potsdam_image_labels.append(0)

for img_path in glob.glob(os.path.join(potsdam_image_path_gable, '*.tif')):
    potsdam_image_paths_list.append(str(img_path))
    potsdam_image_labels.append(1)

for img_path in glob.glob(os.path.join(potsdam_image_path_hip, '*.tif')):
    potsdam_image_paths_list.append(str(img_path)) 
    potsdam_image_labels.append(2)
    
potsdam_image_paths_list.sort()
assert(len(potsdam_image_paths_list) == 300 and len(potsdam_image_labels) == 300)

In [22]:
zipped_list = list(zip(potsdam_image_paths_list, potsdam_image_labels))
random.shuffle(zipped_list)
potsdam_image_paths_list, potsdam_image_labels = zip(*zipped_list)
potsdam_image_paths_list, potsdam_image_labels = list(potsdam_image_paths_list), list(potsdam_image_labels)

print(potsdam_image_paths_list)
print(potsdam_image_labels)

In [23]:
mini_image_paths_list = []
mini_labels = []

for i in range(0,10,1):
    mini_image_paths_list.append(potsdam_image_paths_list[i])
    mini_labels.append(0)
    
for i in range(100,110,1):
    mini_image_paths_list.append(potsdam_image_paths_list[i])
    mini_labels.append(1)
    
for i in range(200,210,1):
    mini_image_paths_list.append(potsdam_image_paths_list[i])
    mini_labels.append(2)
    
print(mini_image_paths_list)
print(mini_labels)

In [24]:
zipped_list = list(zip(mini_image_paths_list, mini_labels))
random.shuffle(zipped_list)
mini_image_paths_list, mini_labels = zip(*zipped_list)
mini_image_paths_list, mini_labels = list(mini_image_paths_list), list(mini_labels)

print(mini_image_paths_list)
print(mini_labels)

In [25]:
# train_X_potsdam = []
# train_Y_potsdam = []
# for i, img in enumerate(mini_image_paths_list):
#     img = cv2.imread(img, cv2.IMREAD_COLOR)
#     resized_img = cv2.resize(img,(256, 256), interpolation = cv2.INTER_CUBIC)
#     train_X_potsdam.append(resized_img)
    
# train_Y_potsdam = mini_labels

train_X_potsdam = []
train_Y_potsdam = []
for i, img in enumerate(potsdam_image_paths_list):
    img = cv2.imread(img, cv2.IMREAD_COLOR)
    resized_img = cv2.resize(img,(256, 256), interpolation = cv2.INTER_CUBIC)
    train_X_potsdam.append(resized_img)
    
train_Y_potsdam = potsdam_image_labels

In [26]:
train_X_potsdam = np.array(train_X_potsdam)
train_Y_potsdam = np.array(train_Y_potsdam)

train_X_potsdam = train_X_potsdam.astype('float32')
train_X_potsdam /= 255.
train_Y_potsdam = train_Y_potsdam.reshape(train_Y_potsdam.shape[0],1)
train_Y_potsdam = to_categorical(train_Y_potsdam, num_classes = 3)

print(train_X_potsdam.shape, train_Y_potsdam.shape)

### **Test Time Augmentation**

In [27]:
# make a prediction using test-time augmentation
def tta_prediction(datagen, model, image, n_examples):
    # convert image into dataset
    samples = expand_dims(image, 0)
    # prepare iterator
    it = datagen.flow(samples, batch_size=n_examples)
    # make predictions for each augmented image
    yhats = model.predict_generator(it, steps=n_examples, verbose=0)
    # sum across predictions
    summed = np.sum(yhats, axis=0)
    # argmax across classes
    return argmax(summed)

In [35]:
# evaluate a model on a dataset using test-time augmentation
def tta_evaluate_model(model, test_X, test_Y):
    # configure image data augmentation
    datagen = ImageDataGenerator( rotation_range = 7,  # randomly rotate images in the range (degrees, 0 to 180)
                                  width_shift_range = 0.10,  # randomly shift images horizontally (fraction of total width)
                                  height_shift_range = 0.10,  # randomly shift images vertically (fraction of total height)
                                  horizontal_flip = True,  # randomly flip images
                                  vertical_flip = True,   # randomly flip images
                                  fill_mode = 'reflect')
    # define the number of augmented images to generate per test set image
    n_examples_per_image = 5
    yhats = list()
    for i in range(len(test_X)):
        # make augmented prediction
        yhat = tta_prediction(datagen, model, test_X[i], n_examples_per_image)
        # store for evaluation
        yhats.append(yhat)
    # calculate accuracy
    print(yhats)
    testY_labels = argmax(test_Y, axis=1)
    acc = accuracy_score(testY_labels, yhats)
    plot_confusion_matrix(testY_labels, yhats)
    print_classification_report(testY_labels, yhats)
    return acc, yhats

### **CNN with RMS Prop**

* Optimizer - RMS Prop
* Learning Rate -0.0001
* Batch size - 16
* Epochs - 50

In [29]:
CNNmodel_RMS = Sequential()

CNNmodel_RMS.add(Conv2D(64, (3, 3), padding='same',
                 input_shape = train_X.shape[1:]))
CNNmodel_RMS.add(Activation('relu'))
CNNmodel_RMS.add(Conv2D(110, (3, 3)))
CNNmodel_RMS.add(Activation('relu'))
CNNmodel_RMS.add(MaxPooling2D(pool_size=(2, 2)))
CNNmodel_RMS.add(Dropout(0.15))

CNNmodel_RMS.add(Conv2D(84, (3, 3), padding='same'))
CNNmodel_RMS.add(Activation('relu'))
CNNmodel_RMS.add(Conv2D(84, (3, 3)))
CNNmodel_RMS.add(Activation('relu'))
CNNmodel_RMS.add(MaxPooling2D(pool_size=(2, 2)))
CNNmodel_RMS.add(Dropout(0.20))

CNNmodel_RMS.add(Conv2D(64, (3, 3), padding='same'))
CNNmodel_RMS.add(Activation('relu'))
CNNmodel_RMS.add(Conv2D(64, (3, 3)))
CNNmodel_RMS.add(Activation('relu'))
CNNmodel_RMS.add(MaxPooling2D(pool_size=(2, 2)))
CNNmodel_RMS.add(Dropout(0.20))

CNNmodel_RMS.add(Conv2D(32, (3, 3), padding='same'))
CNNmodel_RMS.add(Activation('relu'))
CNNmodel_RMS.add(Conv2D(32, (3, 3)))
CNNmodel_RMS.add(Activation('relu'))
CNNmodel_RMS.add(MaxPooling2D(pool_size=(2, 2)))
CNNmodel_RMS.add(Dropout(0.20))

CNNmodel_RMS.add(Flatten())
CNNmodel_RMS.add(Dense(1024))
CNNmodel_RMS.add(Activation('relu'))
CNNmodel_RMS.add(Dense(3))
CNNmodel_RMS.add(Activation('softmax'))

CNNmodel_RMS.summary()


In [30]:
# checkpoint: save best model during the training 
filepath = "./CNN-RMSProp-50epochs"
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 0, save_best_only = True, mode = 'max')
callbacks_list = [checkpoint]

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate = 0.0001, decay = 1e-7)

# Let's train the model using RMSprop
CNNmodel_RMS.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['acc']) 

In [31]:
history_CNN_RMS = CNNmodel_RMS.fit(data_generation.flow(train_X, train_Y, batch_size = 16),
                                steps_per_epoch = train_X.shape[0] // 16,
                                epochs = 50,
                                validation_data = (val_X, val_Y),
                                callbacks = [callbacks_list])

In [32]:
plot_accuracy_loss(history_CNN_RMS)
pred_labels, test_labels = make_predictions(CNNmodel_RMS, test_X, test_Y)
print("Predictions: ", pred_labels)
print("Actual: ", test_labels)
plot_confusion_matrix(test_labels, pred_labels)
print_classification_report(test_labels, pred_labels)
plot_AUC_ROC(CNNmodel_RMS, test_X, test_Y)

### **Testing CNN model on PotsDam dataset**

In [33]:
CNNmodel_RMS = tf.keras.models.load_model('./CNN-RMSProp-50epochs')
test_loss = CNNmodel_RMS.evaluate(train_X_potsdam, train_Y_potsdam)
predictions = CNNmodel_RMS.predict(train_X_potsdam)
pred_labels = np.argmax(predictions, axis = 1)
test_labels = np.argmax(train_Y_potsdam, axis = 1)
print(pred_labels)
print(test_labels)
print(test_loss)
print_classification_report(test_labels, pred_labels)
plot_confusion_matrix(test_labels, pred_labels)

In [36]:
acc, yhats = tta_evaluate_model(CNNmodel_RMS, train_X_potsdam, train_Y_potsdam)
print(acc)

### **ResNet50**

* Optimizer - RMSProp
* Learning rate - 0.00001
* Epochs - 50
* Batch size- 8

In [37]:
ResNet50_model = keras.applications.resnet50.ResNet50()
image_size = 256
input_shape = (image_size, image_size, 3)

pre_trained_model_finetuning = ResNet50(input_shape=input_shape, include_top=False, weights="imagenet")
pre_trained_model_finetuning.trainable = True

In [38]:
last_layer = pre_trained_model_finetuning.get_layer('conv5_block3_out')
last_output = last_layer.output

x = GlobalMaxPooling2D()(last_output)
x = Dense(64, activation = 'relu')(x)
x = Dropout(0.2)(x)

x = Dense(128, activation = 'relu')(x)
x = Dropout(0.2)(x)

x = Dense(256, activation = 'relu')(x)
x = Dropout(0.2)(x)

x = Dense(512, activation = 'relu')(x)
x = Dropout(0.2)(x)

x = keras.layers.Dense(3, activation = 'softmax')(x)

ResNet50_finetuning_bs8 = Model(pre_trained_model_finetuning.input, x)

ResNet50_finetuning_bs8.summary()

In [39]:
# checkpoint: save best model during the training 
filepath = "./ResNet50-50epochs-bs8"
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')
callbacks_list = [checkpoint]

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate = 0.00001, decay = 1e-7)

# Let's train the model using RMSprop
ResNet50_finetuning_bs8.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['acc']) 

In [40]:
history_ResNet50_finetuning_bs8 = ResNet50_finetuning_bs8.fit(data_generation.flow(train_X, train_Y, batch_size = 8),
                                steps_per_epoch = train_X.shape[0] // 8,
                                epochs = 50,
                                validation_data = (val_X, val_Y),
                                callbacks = [callbacks_list])

In [41]:
plot_accuracy_loss(history_ResNet50_finetuning_bs8)
pred_labels, test_labels = make_predictions(ResNet50_finetuning_bs8, test_X, test_Y)
print("Predictions: ", pred_labels)
print("Actual: ", test_labels)
plot_confusion_matrix(test_labels, pred_labels)
print_classification_report(test_labels, pred_labels)
plot_AUC_ROC(ResNet50_finetuning_bs8, test_X, test_Y)

### **ResNet testing on PotsDam Dataset**

In [42]:
ResNet50_finetuning_bs8 = tf.keras.models.load_model('./ResNet50-50epochs-bs8')
test_loss = ResNet50_finetuning_bs8.evaluate(train_X_potsdam, train_Y_potsdam)
predictions = ResNet50_finetuning_bs8.predict(train_X_potsdam)
pred_labels = np.argmax(predictions, axis = 1)
test_labels = np.argmax(train_Y_potsdam, axis = 1)
print(pred_labels)
print(test_labels)
print(test_loss)
print_classification_report(test_labels, pred_labels)
plot_confusion_matrix(test_labels, pred_labels)

In [43]:
acc, yhats = tta_evaluate_model(ResNet50_finetuning_bs8, train_X_potsdam, train_Y_potsdam)
print(acc)

### **EfficientNetB4 with finetuning**

* Optimizer - RMSProp
* Learning rate - 0.00001
* Batch size - 16
* Epochs - 50

In [44]:
image_size = 256
input_shape = (image_size, image_size, 3)
EfficientNetModel = EfficientNetB4(input_shape=input_shape, include_top = False, weights='imagenet')
EfficientNetModel.trainable = True

In [45]:
last_layer = EfficientNetModel.get_layer('top_bn')
last_output = last_layer.output

x = GlobalMaxPooling2D()(last_output)
x = Dense(64, activation = 'relu')(x)
x = Dropout(0.2)(x)

x = Dense(128, activation = 'relu')(x)
x = Dropout(0.2)(x)

x = Dense(256, activation = 'relu')(x)
x = Dropout(0.2)(x)

x = Dense(512, activation = 'relu')(x)
x = Dropout(0.2)(x)

x = keras.layers.Dense(3, activation = 'softmax')(x)

EfficientNetB4_finetuning_RMS_bs16 = Model(EfficientNetModel.input, x)

EfficientNetB4_finetuning_RMS_bs16.summary()

In [46]:
# checkpoint: save best model during the training 
filepath = "./EfficientNetB4-50epochs-bs16-RMS"
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')
callbacks_list = [checkpoint]

# initiate RMS optimizer
opt = keras.optimizers.RMSprop(learning_rate = 0.00001, decay = 1e-7)

# Let's train the model using RMS
EfficientNetB4_finetuning_RMS_bs16.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['acc'])  

In [47]:
history_EfficientNetB4_finetuning_RMS_bs16 = EfficientNetB4_finetuning_RMS_bs16.fit(data_generation.flow(train_X, train_Y, batch_size = 16),
                                steps_per_epoch = train_X.shape[0] // 16,
                                epochs = 50,
                                validation_data = (val_X, val_Y),
                                callbacks = [callbacks_list])

In [48]:
plot_accuracy_loss(history_EfficientNetB4_finetuning_RMS_bs16)
pred_labels, test_labels = make_predictions(EfficientNetB4_finetuning_RMS_bs16, test_X, test_Y)
print("Predictions: ", pred_labels)
print("Actual: ", test_labels)
plot_confusion_matrix(test_labels, pred_labels)
print_classification_report(test_labels, pred_labels)
plot_AUC_ROC(EfficientNetB4_finetuning_RMS_bs16, test_X, test_Y)

### **Testing EfficientNetB4-RMS Prop on PotsDam dataset**

In [49]:
EfficientNetB4_finetuning_RMS_bs16 = tf.keras.models.load_model('./EfficientNetB4-50epochs-bs16-RMS')
test_loss = EfficientNetB4_finetuning_RMS_bs16.evaluate(train_X_potsdam, train_Y_potsdam)
predictions = EfficientNetB4_finetuning_RMS_bs16.predict(train_X_potsdam)
pred_labels = np.argmax(predictions, axis = 1)
test_labels = np.argmax(train_Y_potsdam, axis = 1)
print(pred_labels)
print(test_labels)
print(test_loss)
print_classification_report(test_labels, pred_labels)
plot_confusion_matrix(test_labels, pred_labels)

In [50]:
acc, yhats = tta_evaluate_model(EfficientNetB4_finetuning_RMS_bs16, train_X_potsdam, train_Y_potsdam)
print(acc)

### **EfficientNetB4**

* Optimizer - Adam
* Learning rate - 0.00001
* Batch size - 16
* Epochs - 50

In [51]:
image_size = 256
input_shape = (image_size, image_size, 3)
EfficientNetModel = EfficientNetB4(input_shape=input_shape, include_top = False, weights='imagenet')
EfficientNetModel.trainable = True

In [52]:
last_layer = EfficientNetModel.get_layer('top_bn')
last_output = last_layer.output

x = GlobalMaxPooling2D()(last_output)
x = Dense(64, activation = 'relu')(x)
x = Dropout(0.2)(x)

x = Dense(128, activation = 'relu')(x)
x = Dropout(0.2)(x)

x = Dense(256, activation = 'relu')(x)
x = Dropout(0.2)(x)

x = Dense(512, activation = 'relu')(x)
x = Dropout(0.2)(x)

x = keras.layers.Dense(3, activation = 'softmax')(x)

EfficientNetB4_finetuning_Adam_bs16 = Model(EfficientNetModel.input, x)

In [53]:
# checkpoint: save best model during the training 
filepath = "./EfficientNetB4-50epochs-bs16-Adam"
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')
callbacks_list = [checkpoint]

# initiate RMS optimizer
opt = keras.optimizers.Adam(learning_rate = 0.00001, decay = 1e-7)

# Let's train the model using Adam
EfficientNetB4_finetuning_Adam_bs16.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['acc'])  

In [ ]:
history_EfficientNetB4_finetuning_Adam_bs16 = EfficientNetB4_finetuning_Adam_bs16.fit(data_generation.flow(train_X, train_Y, batch_size = 16),
                                steps_per_epoch = train_X.shape[0] // 16,
                                epochs = 50,
                                validation_data = (val_X, val_Y),
                                callbacks = [callbacks_list])

In [ ]:
plot_accuracy_loss(history_EfficientNetB4_finetuning_Adam_bs16)
pred_labels, test_labels = make_predictions(EfficientNetB4_finetuning_Adam_bs16, test_X, test_Y)
print("Predictions: ", pred_labels)
print("Actual: ", test_labels)
plot_confusion_matrix(test_labels, pred_labels)
print_classification_report(test_labels, pred_labels)
plot_AUC_ROC(EfficientNetB4_finetuning_Adam_bs16, test_X, test_Y)

### **Testing EfficientNetB4-Adam on PotsDam dataset**

In [ ]:
EfficientNetB4_finetuning_Adam_bs16 = tf.keras.models.load_model('./EfficientNetB4-50epochs-bs16-Adam')
test_loss = EfficientNetB4_finetuning_Adam_bs16.evaluate(train_X_potsdam, train_Y_potsdam)
predictions = EfficientNetB4_finetuning_Adam_bs16.predict(train_X_potsdam)
pred_labels = np.argmax(predictions, axis = 1)
test_labels = np.argmax(train_Y_potsdam, axis = 1)
print(pred_labels)
print(test_labels)
print(test_loss)
print_classification_report(test_labels, pred_labels)
plot_confusion_matrix(test_labels, pred_labels)

In [ ]:
acc, yhats = tta_evaluate_model(EfficientNetB4_finetuning_Adam_bs16, train_X_potsdam, train_Y_potsdam)
print(acc)

### **VGG16**

* Optimizer - RMSProp
* Learning rate - 0.00001
* Batch size - 4
* Epochs - 50

In [ ]:
image_size = 256
input_shape = (image_size, image_size, 3)

pre_trained_model_VGG = VGG16(input_shape=input_shape, include_top=False, weights="imagenet")
pre_trained_model_VGG.trainable = True

In [ ]:
last_layer = pre_trained_model_VGG.get_layer('block5_pool')
last_output = last_layer.output

x = GlobalMaxPooling2D()(last_output)
x = Dense(64, activation = 'relu')(x)
x = Dropout(0.2)(x)

x = Dense(128, activation = 'relu')(x)
x = Dropout(0.2)(x)

x = Dense(256, activation = 'relu')(x)
x = Dropout(0.2)(x)

x = Dense(512, activation = 'relu')(x)
x = Dropout(0.2)(x)

x = keras.layers.Dense(3, activation = 'softmax')(x)

VGG16_finetuning_RMS_bs4 = Model(pre_trained_model_VGG.input, x)

VGG16_finetuning_RMS_bs4.summary()

In [ ]:
# checkpoint: save best model during the training 
filepath = "./VGG16-50epochs-RMS-bs4"
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')
callbacks_list = [checkpoint]

# initiate RMSProp optimizer
opt = keras.optimizers.RMSprop(learning_rate = 0.00001, decay = 1e-7)

# Let's train the model using RMSProp
VGG16_finetuning_RMS_bs4.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['acc'])  

In [ ]:
history_VGG16_finetuning_RMS_bs4 = VGG16_finetuning_RMS_bs4.fit(data_generation.flow(train_X, train_Y, batch_size = 4),
                                steps_per_epoch = train_X.shape[0] // 4,
                                epochs = 50,
                                validation_data = (val_X, val_Y),
                                callbacks = [callbacks_list])

In [ ]:
plot_accuracy_loss(history_VGG16_finetuning_RMS_bs4)
pred_labels, test_labels = make_predictions(VGG16_finetuning_RMS_bs4, test_X, test_Y)
print("Predictions: ", pred_labels)
print("Actual: ", test_labels)
plot_confusion_matrix(test_labels, pred_labels)
print_classification_report(test_labels, pred_labels)
plot_AUC_ROC(VGG16_finetuning_RMS_bs4, test_X, test_Y)

### **Testing VGG16 on PotsDam dataset**

In [ ]:
VGG16_finetuning_RMS_bs4 = tf.keras.models.load_model('./VGG16-50epochs-RMS-bs4')
test_loss = VGG16_finetuning_RMS_bs4.evaluate(train_X_potsdam, train_Y_potsdam)
predictions = VGG16_finetuning_RMS_bs4.predict(train_X_potsdam)
pred_labels = np.argmax(predictions, axis = 1)
test_labels = np.argmax(train_Y_potsdam, axis = 1)
print(pred_labels)
print(test_labels)
print(test_loss)
print_classification_report(test_labels, pred_labels)
plot_confusion_matrix(test_labels, pred_labels)

In [ ]:
acc, yhats = tta_evaluate_model(VGG16_finetuning_RMS_bs4, train_X_potsdam, train_Y_potsdam)
print(acc)